In [1]:
import time
import copy
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import pyecharts.options as opts
from pyecharts.charts import Funnel

In [2]:
%matplotlib inline
plt.rcParams["font.sans-serif"]=["SimHei"]     #用来正常显示中文标签
plt.rcParams["axes.unicode_minus"] = False   #用来正常显示负号
plt.style.use("ggplot")

1. 导入原始数据：

In [3]:
dt = pd.read_csv('UserBehavior.csv',
                     names=['user_id', 'item_id', 'category_id', 'behavior_type', 'timestamp'])
print('原始数据shape',dt.shape)

原始数据shape (100150807, 5)


2. 为了方便只取 1% 的数据，后续做正式训练的时候把这一部分注释掉

In [4]:
dt = dt.sample(frac=0.01)
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001508 entries, 73509324 to 70178650
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   user_id        1001508 non-null  int64 
 1   item_id        1001508 non-null  int64 
 2   category_id    1001508 non-null  int64 
 3   behavior_type  1001508 non-null  object
 4   timestamp      1001508 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 45.8+ MB


3. 数据清洗

    (1) 缺失值处理,结果显示数据没有缺失值，比较完整干净。

In [5]:
dt.isnull().sum()

user_id          0
item_id          0
category_id      0
behavior_type    0
timestamp        0
dtype: int64

(2) 重复值处理

In [6]:
dt.duplicated().sum()

0

In [7]:
dt.dropna(axis=0, how='any', inplace=True)
print("去除重复值的shape", dt.shape)

去除重复值的shape (1001508, 5)


(3) 去除异常值，即时间戳不在规定的时间内

In [8]:
# 保留时间在2017-11-25 00:00:00至2017-12-4 00:00:00之间的数据，合计9天
dt = dt[(dt['timestamp'] >= 1511539200) & (dt['timestamp'] < 1512316800)]
print('时间段筛选后数据shape：',dt.shape)

时间段筛选后数据shape： (1000932, 5)


4. 按时间升序排列

In [9]:
dt = dt.sort_values(['timestamp','user_id']).reset_index(drop=True)

5. 时间维度拆解，增添新列(将时间戳扩充为日期和小时)

In [10]:
date_times = []
dates = []
hours = []
for t in dt['timestamp']:
    date_time = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t))
    date_times.append(date_time)
    date_ = time.strftime('%Y-%m-%d',time.localtime(t))
    dates.append(date_)
    hour = time.strftime('%H',time.localtime(t))
    hours.append(hour)
dt['datetime'] = date_times
dt['date'] = dates
dt['hour'] = hours
dt.head()

,user_id,item_id,category_id,behavior_type,timestamp,datetime,date,hour
0,698434,2272322,874415,pv,1511539200,2017-11-25 00:00:00,2017-11-25,00
1,312258,4338016,3758209,pv,1511539201,2017-11-25 00:00:01,2017-11-25,00
2,473381,1404187,2344159,pv,1511539201,2017-11-25 00:00:01,2017-11-25,00
3,161104,2970300,4217906,pv,1511539202,2017-11-25 00:00:02,2017-11-25,00
4,260997,2480849,2851989,pv,1511539202,2017-11-25 00:00:02,2017-11-25,00


6. 以日期为单位做宏观分析

In [11]:
dt_pv = dt[dt['behavior_type']=='pv'][['user_id','date']].groupby(['date']).count()
dt_pv.rename(columns={"user_id":"浏览数"},inplace=True)

dt_fav = dt[dt['behavior_type']=='fav'][['user_id','date']].groupby(['date']).count()
dt_fav.rename(columns={"user_id":"收藏数"},inplace=True)

dt_cart = dt[dt['behavior_type']=='cart'][['user_id','date']].groupby(['date']).count()
dt_cart.rename(columns={"user_id":"加购数"},inplace=True)

dt_buy = dt[dt['behavior_type']=='buy'][['user_id','date']].groupby(['date']).count()
dt_buy.rename(columns={"user_id":"购买数"},inplace=True)

dt_sta_date = dt_pv.join([dt_fav,dt_cart,dt_buy])

dt_sta_date

,浏览数,收藏数,加购数,购买数
date,,,,
2017-11-25,93935,3117,5650,2042
2017-11-26,95993,3129,5757,2029
2017-11-27,90364,2889,5470,2285
2017-11-28,88110,2971,5370,2114
2017-11-29,91923,2922,5459,2227
2017-11-30,93827,3035,5684,2199
2017-12-01,97329,3024,6216,2059
2017-12-02,123376,4036,7919,2514
2017-12-03,121755,3905,7732,2566


7. 以小时为单位做宏观分析

In [12]:
dt_pv = dt[dt['behavior_type']=='pv'][['user_id','hour']].groupby(['hour']).count()
dt_pv.rename(columns={"user_id":"浏览数"},inplace=True)

dt_fav = dt[dt['behavior_type']=='fav'][['user_id','hour']].groupby(['hour']).count()
dt_fav.rename(columns={"user_id":"收藏数"},inplace=True)

dt_cart = dt[dt['behavior_type']=='cart'][['user_id','hour']].groupby(['hour']).count()
dt_cart.rename(columns={"user_id":"加购数"},inplace=True)

dt_buy = dt[dt['behavior_type']=='buy'][['user_id','hour']].groupby(['hour']).count()
dt_buy.rename(columns={"user_id":"购买数"},inplace=True)

dt_sta_hour = dt_pv.join([dt_fav,dt_cart,dt_buy])

dt_sta_hour

,浏览数,收藏数,加购数,购买数
hour,,,,
00,31029,1042,1787,622
01,14207,523,900,256
02,7740,300,493,110
03,5249,183,363,72
04,4477,137,296,61
05,5813,205,337,70
06,12144,436,819,178
07,22127,741,1415,360
08,30334,1095,1939,661
